TRABAJO FIN DE MASTER

Máster en Big Data y Data Science

Título: Descubriendo similitud entre documentos a partir de entidades nombradas

Tutor: Pablo A. Haya Coll

Estudiante: Emilio J. Macias Macias

# Fase 3: Persistencia y Visualización

In [ ]:
import persistence, visualization
import os
from pathlib import Path

import spacy
from elasticsearch import Elasticsearch
from elasticsearch import helpers

# Carga un modelo Spacy del disco local (debe haber sido entrenado y generado anteriormente).
def load_model(tfm_dir, data_size, epoch, dropout, revision_rate):    
    model = spacy.load(tfm_dir + '/models/model_' + data_size + '_e' + str(epoch) +
                       '_d' + str(dropout) + '_r' + str(revision_rate))
    return model

## Persistencia de datos
Los términos enriquecidos y las entidades nombradas de cada noticia se almacenarán en índices de Elastic Search (ES). Será preciso que la anotación siga el formato Brat (http://brat.nlplab.org/standoff.html) de manera que facilite la visualización posterior en caso de que sea necesario (http://brat.nlplab.org/examples.html).

La organización que se escoja deberá permitir recuperar todas las noticias que compartan un determinado término o una determinada entidad nombrada. Deberá ser posible calcular estadísticas agregando las diferentes características lingüí­sticas como los lemas, el pos, o las relaciones de dependencias.


In [ ]:
# Especificar parámetros para cargar el modelo actualizado:

# Directorio raiz del proyecto.
tfm_dir = str(Path(os.getcwd()).parent)

model_size = 'medium' # small | medium | large
epoch = 4 # 1 | 5 | 10
dropout = 0.5 # 0.25 | 0.5 | 0.75
revision_rate = 0.7 # 0.25 | 0.5 | 0.75

# Cargar modelo actualizado.
updated_model = load_model(tfm_dir, model_size, epoch, dropout, revision_rate)

In [ ]:
# Directorio de las noticias a indexar.
data_size = 'small' # small | medium | large
data_dir = tfm_dir + '/data/' + data_size + '-data'

# Índice noticias para ES
newsIndex = "news-index"

# Duplicar información para poder agregar entidades en ES
# (Kibana no soporta objetos anidados)
entityIndex = "entity-index"

# Información para agregar características linguísticas
# como lemas y parts-of-speech.
lingIndex = "ling-index"

# Establecer comunicación con ES
es_client = Elasticsearch(hosts = [{'host':'localhost', 'port':9200}])

In [ ]:
# Procesar entidades e información lingüística de las noticias para indexar en ElasticSearch.
actions, entities_by_news = persistence.process_news(data_dir, updated_model, newsIndex, entityIndex, lingIndex)

# Cargar todos los documentos en ES.
helpers.bulk(es_client, actions)

## Visualización de resultados
Se habilitara un interfaz de usuario, mediante Kibana, que permita mostrar las entidades detectadas, recuperar los textos donde aparece una entidad o una palabra, así como información básica del corpus sobre palabras, lemas, y oraciones.

Se implementara un algoritmo de distancia de documentos en función de las entidades que compartan, y se mostrara un grafo de todos los documentos (networkx, igraph o d3.js). Opcionalmente, se colorearan automáticamente los documentos mediante algoritmo de comunidades. Opcionalmente se podrá buscar un documento y visualizar su red de documentos similares.

In [ ]:
# Modelo de vectores (inglés). Usado para detectar similitud entre listas de entidades nombradas.
nlp_vectors = spacy.load('en_vectors_web_lg')

# Calcular matriz de distancias entre documentos.
dist_matrix, edges = visualization.find_doc_distances(entities_by_news, nlp_vectors)

# Filtrar por tamaño de comunidades, para representar solamente las que tengan un tamaño por encima
# del umbral (% mínimo de noticias sobre el total).
threshold = 0.05

# Crear grafo de noticias y comunidades.
visualization.plot_graph(dist_matrix, edges, threshold, entities_by_news, tfm_dir)